# BioExplorer - Vasculature
![](../bioexplorer_vasculature_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer
import seaborn as sns

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

### Load vasculature

In [ ]:
vasculature_assembly_name = 'Vasculature'
vasculature_population_name = 'vasculature'
be.remove_assembly(vasculature_assembly_name)
vasculature_assembly = be.add_assembly(vasculature_assembly_name)
vasculature_model = be.add_vasculature(
    assembly_name=vasculature_assembly_name,
    population_name=vasculature_population_name,
    use_sdf=True,
    quality=be.GEOMETRY_QUALITY_MEDIUM,
    sql_filter='sqrt(pow(x - 345, 2) + pow(y - 1359, 2) + pow(z - 600, 2)) < 100'
)

In [ ]:
status = core.set_renderer(
    current='bio_explorer',
    subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 1000.0
params.shadows = 1.0
params.soft_shadows = 1.0
status = core.set_renderer_params(params)

In [ ]:
astrocyte_assembly_name = 'Astrocyte'
astrocyte_population_name = 'Astrocyte'
be.remove_assembly(astrocyte_assembly_name)
astrocyte_assembly = be.add_assembly(astrocyte_assembly_name)
astrocyte_model = be.add_astrocytes(
    assembly_name=astrocyte_assembly_name,
    population_name=astrocyte_population_name,
    vasculature_population_name=vasculature_population_name,
    population_color_scheme=be.POPULATION_COLOR_SCHEME_ID,
    use_sdf=True, sql_filter='guid=6976'
)

In [ ]:
def set_materials(
    model_id, material_ids, palette, shading_mode=be.SHADING_MODE_NONE,
    cast_user_data=False, reflection_index=0.0, opacity=1):

    user_params=list()
    opacities=list()
    refraction_indices=list()
    reflection_indices=list()
    shading_modes=list()
    diffuse_colors=list()
    specular_colors=list()
    specular_exponents=list()
    glossinesses=list()
    emissions=list()
    cast_user_datas = list()
    
    for i in range(len(palette)):
        user_params.append(0.01)
        opacities.append(opacity)
        refraction_indices.append(0.9)
        reflection_indices.append(reflection_index)
        specular_exponents.append(50.0)
        glossinesses.append(0.9)
        c=palette[i]
        diffuse_colors.append(c)
        specular_colors.append(c                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        )
        shading_modes.append(shading_mode)
        emissions.append(0)
        cast_user_datas.append(cast_user_data)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        user_parameters=user_params,
        opacities=opacities, reflection_indices=reflection_indices,
        shading_modes=shading_modes,
        diffuse_colors=diffuse_colors, specular_colors=specular_colors,
        specular_exponents=specular_exponents, glossinesses=glossinesses,
        emissions=emissions, refraction_indices=refraction_indices,
        cast_user_datas=cast_user_datas)

In [ ]:
def set_scene_materials():
    vasculature_info = be.get_vasculature_info(assembly_name=vasculature_assembly_name)
    vasculature_model_id = vasculature_info['modelId']    
    for model_id in be.get_model_ids()['ids']:
        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)
        if model_id == vasculature_model_id:
            palette = list()
            for i in range(nb_materials):
                palette.append((1.0, 0.0, 0.0))
            set_materials(
                shading_mode=be.SHADING_MODE_NONE,
                model_id=model_id, material_ids=material_ids,
                palette=palette, cast_user_data=False, reflection_index=0.0)
        else:
            palette = sns.color_palette('Blues', 10)
            p = list()
            for i in range(nb_materials):
                p.append(palette[5])
            set_materials(
                shading_mode=be.SHADING_MODE_NONE,
                model_id=model_id, material_ids=material_ids,
                palette=p, cast_user_data=False)
    status = core.set_renderer()

In [ ]:
set_scene_materials()

## Snapshots

In [ ]:
status = core.set_camera(current='bio_explorer_perspective')

### Default overview

##### Single astrocyte

In [ ]:
core.set_camera(
    orientation=[-0.026381403274141492, 0.9989770953600118, 0.036579691219384786,  0.003272720222589006],
    position=[348.0229087689656, 1380.1739741996078, 240.6439526719328],
    target=[346.3668479043361, 1353.8501781666782, 599.0084099562833]
)

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
mm.create_snapshot(
    renderer='bio_explorer',
    path='/tmp', base_name='vasculature_and_single_astrocyte_4k_v1',
    size=[4000,4000], samples_per_pixel=64)

#### Several astrocytes

In [ ]:
astrocytes_assembly_name = 'Astrocytes'
astrocytes_population_name = 'Astrocyte'
be.remove_assembly(astrocytes_assembly_name)
astrocytes_assembly = be.add_assembly(astrocytes_assembly_name)
astrocytes_model = be.add_astrocytes(
    assembly_name=astrocytes_assembly_name,
    population_name=astrocytes_population_name,
    vasculature_population_name=vasculature_population_name,
    population_color_scheme=be.POPULATION_COLOR_SCHEME_ID,
    use_sdf=True, 
    sql_filter = 'sqrt(pow(x - 345, 2) + pow(y - 1359, 2) + pow(z - 600, 2)) < 80 and guid!=6976'
)

In [ ]:
set_scene_materials()

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
core.set_camera(
    orientation=[-0.0263814032741415, 0.9989770953600118, 0.03657969121938479, 0.0032727202225888916],
    position=[348.0781750273631, 1381.052455023975, 228.68457175797897],
    target=[346.3668479043361, 1353.8501781666782, 599.0084099562833],    
)

In [ ]:
mm.create_snapshot(
    renderer='bio_explorer',
    path='/tmp', base_name='vasculature_and_few_astrocytes_4k_v1',
    size=[4000,4000], samples_per_pixel=64)

## Movie with simulated radii on vasculature

In [ ]:
from IPython.display import clear_output
output_folder = '/tmp'

nb_frames = 100
report_simulation_id = 2

for frame in range(nb_frames):
    clear_output()
    print('Frame %d/%d' % (frame, nb_frames))
    # Set simulation frame
    core.set_animation_parameters(current=frame)

    # Apply radii changes on vasculature
    be.set_vasculature_radius_report(
        assembly_name=vasculature_assembly_name,
        population_name=vasculature_population_name,
        report_simulation_id=report_simulation_id,
        frame=frame
    )
    # Apply radii changes on single astrocyte
    be.set_vasculature_radius_report(
        assembly_name=astrocyte_assembly_name,
        population_name=vasculature_population_name,
        report_simulation_id=report_simulation_id,
        frame=frame
    )
    # Apply radii changes on other astrocytes
    be.set_vasculature_radius_report(
        assembly_name=astrocytes_assembly_name,
        population_name=vasculature_population_name,
        report_simulation_id=report_simulation_id,
        frame=frame
    )
    # Apply materials
    set_scene_materials()

    # Render frame
    mm.create_snapshot(
        renderer='bio_explorer',
        path=output_folder, base_name='%05d' % frame,
        size=[4096, 4096], samples_per_pixel=64)